In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from chainconsumer import ChainConsumer
import os


/home/shih/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
from utils import loglin_transform

In [3]:
from model import EmbeddingLayer, define_model
import torch.nn as nn


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]='0,1,2'


CUDA = True
device = torch.device("cuda:0" if CUDA else "cpu")

kwargs = {'num_workers': 4, 'pin_memory': True} if CUDA else {}
kwargs = {}


In [5]:
ntoalist=[405, 444, 299, 233, 259, 275, 133, 265, 167, 460]

embedding=EmbeddingLayer(ntoalist,noutput=50)

#(ninputlist=(np.array(ntoalist)).tolist(),noutput=20,nsingle=20)
model=define_model(nfeatures=22,nhidden=2,hidden_size=200,embedding=embedding,dropout=0,nembedding=50,device=device)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
####    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    embedding = nn.DataParallel(embedding)
#####    logprob_parallel=nn.DataParallel(model.log_prob)
    model = nn.DataParallel(model)
embedding=embedding.to(device)
model=model.to(device)



/home/shih/work/ML4GW/flow_posterior/model.py:222: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.output.weight)
/home/shih/work/ML4GW/flow_posterior/model.py:339: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.output.weight)


50
Flow_with_forward(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        (autoregressive_net): MADE(
          (initial_layer): MaskedLinear(in_features=22, out_features=200, bias=True)
          (context_layer): Linear(in_features=50, out_features=200, bias=True)
          (blocks): ModuleList(
            (0): MaskedFeedforwardBlock(
              (linear): MaskedLinear(in_features=200, out_features=200, bias=True)
              (dropout): Dropout(p=0, inplace=False)
            )
            (1): MaskedFeedforwardBlock(
              (linear): MaskedLinear(in_features=200, out_features=200, bias=True)
              (dropout): Dropout(p=0, inplace=False)
            )
          )
          (final_layer): MaskedLinear(in_features=200, out_features=506, bias=True)
        )
      )
      (1): ReversePermutation()
      (2): MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        (autore

In [6]:
testpoints=np.load('testpoints_fLDeltaf0.01_10p_withRN_forsteve_newRN_proj.npy')
testpoints_transf=loglin_transform(testpoints*1e7,1000)


In [7]:
parmin=np.array([-18,1,-19,-19,-19,-19,-19,-19,-19,-19,-19,-19,1,1,1,1,1,1,1,1,1,1])
parmax=np.array([-13,7,-13,-13,-13,-13,-13,-13,-13,-13,-13,-13,7,7,7,7,7,7,7,7,7,7])


In [16]:
# sample events from it for each test point, to evaluate true LL on hexfarm

## August 7 2023 Upgrade this to 1M points from 100k
epochbest=87
model.load_state_dict(torch.load('trained_models/babydataset_1M_fLDeltaf0.01_10p_withRN_SinglePulsarStackedLSTM_v2/babydataset_fLDeltaf0.01_epoch_'+str(epochbest)+'_Model.par'))


flowsamples=[]
likelihoods=[]
model.eval()
with torch.no_grad():


    for ii in range(10):
        print(ii)
        testpoint_transf=testpoints_transf[ii]
        testposterior=torch.empty((0))
        print('Sampling...')
        for isamplebatch in range(10):
            print(isamplebatch)
            testposterior_batch=model.module.sample(100000,
                                  torch.from_numpy(testpoint_transf.reshape((1,2940)).astype('float32')).to(device)\
                                   ).detach().cpu()
            testposterior=torch.cat((testposterior,testposterior_batch))
        testposterior=testposterior.reshape((-1,22))
        
        print('Evaluating flow logprobs...')
        ll_tp=[]
        for ibatch in range(100):
            print(ibatch)
            ll=model(testposterior[ibatch*10000:(ibatch+1)*10000],\
                                   torch.from_numpy(np.tile(testpoint_transf,(10000,1)).astype('float32')).to(device))
        
            ll_tp+=ll.tolist()
        likelihoods.append(ll_tp)
        
        testposterior=testposterior.cpu().detach().numpy()

        datatry=testposterior.reshape((1000000,-1))
        #parameters_transf=-1+2*(parameters-(np.array([-17,3])))/(np.array([-13,6])-np.array([-17,3]))
        datatry=0.5*(parmin + parmax + (parmax-parmin)*datatry)
    #    datatry2=datatry[(datatry[:,1]>2.5) & (datatry[:,1]<7) & \
    #                    (datatry[:,0]>-16.5) & (datatry[:,0]<-13.5)]
        
        flowsamples.append(datatry)
#        c = ChainConsumer()
#        c.add_chain(np.dstack((datatry[:,1],datatry[:,0]))[0], parameters=["$\gamma_{GW}$", "$log_{10}(A_{GW})$"])
    #    c.add_chain(ll_pars_withRN2[itestpoint][:,[1,0]],weights=ll_vals_withRN_normed[itestpoint],\
    #        parameters=["$\gamma_{GW}$", "$log_{10}(A_{GW})$"],name='5 pulsars w RN')
#        c.add_chain(testpoints_pars[ii][:,[1,0]],weights=testpoints_weights[ii],\
#            parameters=["$\gamma_{GW}$", "$log_{10}(A_{GW})$"],name='reweighted')
#        c.plotter.plot(display=True,figsize="column", truth=[13/3,-15])


0
Sampling...
0
1
2
3
4
5
6
7
8
9
Evaluating flow logprobs...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
1
Sampling...
0
1
2
3
4
5
6
7
8
9
Evaluating flow logprobs...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
2
Sampling...
0
1
2
3
4
5
6
7
8
9
Evaluating flow logprobs...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
8

In [17]:
flowsamples=np.array(flowsamples)
likelihoods=np.array(likelihoods)

In [18]:
flowsamples_dict={}
flowsamples_dict['samples']=flowsamples
flowsamples_dict['residuals']=testpoints
flowsamples_dict['likelihoods']=likelihoods

In [19]:
np.save('testpoints_fLDeltaf0.01_10p_withRN_forsteve_newRN_SinglePulsarStackedLSTM_v2_flowsamples_1M.npy',flowsamples_dict,allow_pickle=True)



# Also try uniform sampling from the prior

In [20]:
#priorsamples=np.random.uniform(size=(10,100000,22),low=parmin,high=parmax)

priorsamples=np.random.uniform(size=(10,1000000,22),low=parmin,high=parmax)

In [19]:
1/np.prod(parmax-parmin)

9.117037425970844e-18

In [21]:
flowsamples_dict={}
flowsamples_dict['samples']=priorsamples
flowsamples_dict['residuals']=testpoints
flowsamples_dict['likelihoods']=np.log(np.ones((10,1000000))/np.prod(parmax-parmin))

In [22]:
np.save('testpoints_fLDeltaf0.01_10p_withRN_forsteve_newRN_UniformPrior_flowsamples_1M.npy',flowsamples_dict,allow_pickle=True)
